In [7]:
import db_utils
import yaml, pandas as pd

with open('credentials.yaml') as file:
    credentials_dict = yaml.safe_load(file)
    
credentials = db_utils.RDSDatabaseConnector(credentials_dict)
loan_payments = credentials.initialise_database()

credentials.save_file(loan_payments)
database = pd.read_csv('new_file.csv', index_col = 'id')

cleaned_data = db_utils.DataTransform(database)

date_data = ['issue_date', 'earliest_credit_line', 'last_payment_date', 'next_payment_date',
             'last_credit_pull_date']

categorical_data = ['member_id', 'term', 'grade', 'sub_grade', 'employment_length', 'home_ownership', 'verification_status', 'loan_status', 
                    'payment_plan', 'purpose', 'policy_code', 'application_type']

non_numeric_data = date_data + categorical_data 
column_headings = database.columns.values.tolist()
numeric_data = [column for column in column_headings if column not in non_numeric_data]

for date_column in date_data:
    database, date_column = cleaned_data.date_data(database, date_column)

for categories in categorical_data:
    database, categories = cleaned_data.change_type(database, categories, 'category')

df_info = db_utils.DataFrameInfo(database)

for column in column_headings:
    null_vals, null_percentage = df_info.missing(database, column)
    null_percentage = round(null_percentage, 2)
    if null_percentage > float(50):
        database = database.drop(column, axis=1)
    elif float(0) < null_percentage < float(50):
        print(f'{column}: {null_vals} null values, {null_percentage}%')

mths_since_last_delinq: 31002 null values, 57.17%
(54231, 41)
mths_since_last_record: 48050 null values, 88.6%
(54231, 40)
next_payment_date: 32608 null values, 60.13%
(54231, 39)
mths_since_last_major_derog: 46732 null values, 86.17%
(54231, 38)
<class 'pandas.core.frame.DataFrame'>
Index: 54231 entries, 38676116 to 72323
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   member_id                   54231 non-null  category      
 1   loan_amount                 54231 non-null  int64         
 2   funded_amount               51224 non-null  float64       
 3   funded_amount_inv           54231 non-null  float64       
 4   term                        49459 non-null  category      
 5   int_rate                    49062 non-null  float64       
 6   instalment                  54231 non-null  float64       
 7   grade                       54231 non-null  category      
 8